In [ ]:
# !/usr/bin/python3


### 1. Data import

import sys  ## system
import numpy as np  ## Matrix Calculate
import glob  ## global variable
import random  ## random sample
from math import exp, gamma, log, sqrt  ## exp,log,sqrt cal
import scipy.stats as ss
import time
from copy import deepcopy as dc
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import logging
from Utils.Utils import *
import csv
import os
import pandas as pd
import numpy as np
from scipy.linalg import cholesky

# path_psig, path_gamg, path_fg, path_ gg, path_bhg, path_ghg C path_deltag, path_llg


###### Important Options
np.random.seed(1028)
UPDATE = False  # <- update from last MCMC
FIX = True  # <- if FIX, then fix the sigma.a, omega.b (sigma, delta) as 1.
VV = 1
# Total Iteration = burnin + mcmc
burnin = 0
mcmc = 200000
target_mh = 0.25

SIG = 1

if sys.argv[1] == "soda":
    PATH = "carb/"
else:
    PATH = "yogurt/"
ff = sys.argv[1]

if sys.argv[2] == '0': MODEL = MODEL + '_FIX'
MODEL = "SSDF_Simu"

PATH = "sims6/s%i/" % int(sys.argv[2])  # <- Simulation
ff = "sims6"

if UPDATE:
    FN = PATH + MODEL + "_update.npz"
    LOGFILE = "Logs/%s_%s_update.log" % (MODEL, ff)
else:
    FN = PATH + MODEL + ".npz"
    LOGFILE = "Logs/sims6/%s_%s_%s.log" % (MODEL, ff, sys.argv[2])

logging.basicConfig(level=logging.INFO,
                    filename=LOGFILE,
                    format="%(asctime)s %(levelname)-7s %(message)s")
with open(LOGFILE, "w"):
    pass

if not os.path.isdir(PATH + MODEL):
    os.mkdir(PATH + MODEL)
if not os.path.isdir(PATH + MODEL + '/MCMC'):
    os.mkdir(PATH + MODEL + '/MCMC')
if not os.path.isdir('Logs'):
    os.mkdir('Logs')

PATH_PSIG = PATH + MODEL + '/MCMC/psig.csv'
PATH_GAMG = PATH + MODEL + '/MCMC/gamg.csv'
PATH_FG = PATH + MODEL + '/MCMC/fg.csv'
PATH_GG = PATH + MODEL + '/MCMC/gg.csv'
PATH_BHG = PATH + MODEL + '/MCMC/bhg.csv'
PATH_GHG = PATH + MODEL + '/MCMC/ghg.csv'
PATH_DELTAG = PATH + MODEL + '/MCMC/deltag.csv'
PATH_SIGMAG = PATH + MODEL + '/MCMC/sigmag.csv'
PATH_LLG = PATH + MODEL + '/MCMC/llg.csv'
PATH_PHIG = PATH + MODEL + '/MCMC/phig.csv'
PATH_VBG = PATH + MODEL + '/MCMC/vbg.csv'

PATH_VBG = PATH + MODEL + '/MCMC/vbg.csv'
PATH_VGAM = PATH + MODEL + '/MCMC/vgam.csv'
PATH_GAMMU = PATH + MODEL + '/MCMC/gammu.csv'
PATH_GAMMUH = PATH + MODEL + '/MCMC/gammuh.csv'
PATH_VPSI = PATH + MODEL + '/MCMC/vpsi.csv'
PATH_PSIMU = PATH + MODEL + '/MCMC/psimu.csv'
PATH_PSIMUH = PATH + MODEL + '/MCMC/psimuh.csv'

PATH_BETAG = PATH + MODEL + '/MCMC/betag.csv'
PATH_KG = PATH + MODEL + '/MCMC/kg.csv'

with open(PATH_PSIG, 'w') as f:
    pass
with open(PATH_GAMG, 'w') as f:
    pass
with open(PATH_FG, 'w') as f:
    pass
with open(PATH_GG, 'w') as f:
    pass
with open(PATH_BHG, 'w') as f:
    pass
with open(PATH_GHG, 'w') as f:
    pass
with open(PATH_DELTAG, 'w') as f:
    pass
with open(PATH_SIGMAG, 'w') as f:
    pass
with open(PATH_LLG, 'w') as f:
    pass
with open(PATH_PHIG, 'w') as f:
    pass
with open(PATH_VBG, 'w') as f:
    pass

with open(PATH_VGAM, 'w') as f:
    pass
with open(PATH_GAMMU, 'w') as f:
    pass
with open(PATH_GAMMUH, 'w') as f:
    pass
with open(PATH_VPSI, 'w') as f:
    pass
with open(PATH_PSIMU, 'w') as f:
    pass
with open(PATH_PSIMUH, 'w') as f:
    pass

with open(PATH_KG, 'w') as f:
    pass
with open(PATH_BETAG, 'w') as f:
    pass

list_of_files = sorted(glob.glob(PATH + 'demand/*.csv'))
list_of_files2 = sorted(glob.glob(PATH + 'prices/*.csv'))
# C = pd.read_csv(PATH + "c.csv").iloc[:, 2:]

## Remove last one day
demand = [np.loadtxt(f, skiprows=1, delimiter=",") for f in list_of_files]
prices = [np.loadtxt(f, skiprows=1, delimiter=",") for f in list_of_files2]

# demand = [np.loadtxt(f, skiprows=1, delimiter=",") for f in list_of_files[:50]]
# prices = [np.loadtxt(f, skiprows=1, delimiter=",") for f in list_of_files2[:50]]

demand = [d[:, :] for d in demand]
prices = [p[:, :] for p in prices]


# demand = [d[:-1, :] for d in demand]
# prices = [p[:-1, :] for p in prices]


#####################  2.  Fucntions
###prior : 0 = pu0 , 1 = pv0, 2 = gu0 , 3 = gv0 , 4 = tpp , 5 = tpg


def rwmh(de, pr, x, ll, alpha, f, psi, g, kt, r, betai, zphi, bh, gh, abh, sigma, delta, vbi, tpa, tpp,
         llb1, sdm, gammu, vgam, gamm, psimu, vpsi, psim, household):
    ## alpha <- gamma   ## a <- alpha

    k = de.shape[1]  ###
    kp = k - 1

    af = f.dot(gh.T)  # <- gh.T like t(gh) in R
    bg = g.dot(bh.T)

    af2 = af + gammu
    bg2 = bg + psimu

    n = psi.shape[0]

    ite = np.zeros((n, 2))  ### num of mcmc iteration
    ## sampling psi,gamma

    #"""
    for i in range(n):

        ## -------------
        ## M-H Sampling

        rpsi = psi[i:(i + 1)].dot(C.T).reshape(-1)
        rpsi[-1] = 0
        rnalpha = alpha[i:(i + 1)].dot(C.T).reshape(-1)


        llb = ll[i] +  logpdf(alpha[i, :], af2[i, :], sigma)
        nalpha = alpha[i] + mvn(np.identity(ck) * tpa[i])
        rnalpha = nalpha.reshape(1, -1).dot(C.T).reshape(-1)

        nll = mdcev(rpsi, rnalpha, de[i, :], pr[i, :], SIG)
        nllb = nll + logpdf(nalpha, af2[i, :], sigma  )

        if nllb - llb > np.log(random.uniform(0, 1)):
            alpha[i] = nalpha
            ll[i] = nll
            llb1[i] = nllb
            ite[i, 0] += 1
        else:
            rnalpha = alpha[i:(i + 1)].dot(C.T).reshape(-1)

        llb = ll[i] + logpdf(psi[i], bg2[i], delta)

        npsi = psi[i] + mvn(np.identity(ck) * tpp[i])
        rnpsi = npsi.reshape(1, -1).dot(C.T).reshape(-1)
        rnpsi[-1] = 0
        nll = mdcev(rnpsi, rnalpha, de[i, :], pr[i, :], SIG)
        nllb = nll + logpdf(npsi, bg2[i, :], delta)

        if nllb - llb > np.log(random.uniform(0, 1)):
            psi[i] = npsi.transpose()
            ll[i] = nll
            llb1[i] = nllb
            ite[i, 1] += 1
    #"""

    # gammu
    y = alpha - af
    y2 = psi - bg

    for i in range(1, ck):
        gammu[i] = bmreg1(y[:, i:(i + 1)], np.ones((n, 1)), None, 1, gamm[i], vgam[i, i], e=False)
    for i in range(ck):
        psimu[i] = bmreg1(y2[:, i:(i + 1)], np.ones((n, 1)), None, 1, psim[i], vpsi[i, i], e=False)

    #psimu[:] = 0
    #gammu[:] = 0

    ### f & g
    gmat = np.zeros((kl, kl))
    gmat[0, 0] = 1
    gmat[1:, 0] = betai
    #gmat = np.eye(3)

    w = np.identity(3) #* (0.01 ** 2)
   
    r = 0
    if ck == 12:
        theta, kt = FFBS(np.concatenate([alpha - gammu, psi - psimu], axis=1), abh, sdm , gmat, w, m0, c0, kt, r)
    else:
        theta, kt = FFBS(np.concatenate([alpha - gammu, (psi - psimu)[:,:-1]], axis=1), abh, sdm, gmat, w, m0, c0, kt, r)

    f = theta[:, 0:(1)]
    g = theta[:, 1:(1 + 2)]

    ## delta_ht
    
    xf = f[:-1][kt[1:] == 1] # Collect f_h(t-1) when f_h(t-1) > 0
    for j in range(2):
        # yg = g[kt == 1, j]
        yg = g[1:][kt[1:] == 1, j] # collect g_h(t) when f_h(t-1) > 0

        if yg.shape[0] > 0:
            bs = np.linalg.inv(vbi[j, j] + xf.T.dot(xf) / vg[j, j])  
            bm = bs.dot(vbi[j, j] * zphi[j] + (xf.T.dot(yg)) / vg[j, j])

        else: # when all f_h(t) < 0, then sample from delta_bar
            bs = [1 / (vbi[j, j])]
            bm = zphi[j]

        betai[j] = bm + mvn([bs])
    

    return [alpha, f, psi, g, ll, ite, kt, betai, gammu, psimu, llb1]


def FFBS(data, F, V, G, W, m0, c0, kt, r):
    # W system var
    # F True_t = F * True_t-1 + u +G * w
    t = data.shape[0]
    m = data.shape[1]
    p = F.shape[1]
    m = int(p * (p + 1) / 2)
    mt = np.zeros((t, p))
    ct = np.zeros((t, m))

    m_t = m0.copy()
    Ct = c0.copy()
    k_t = 1
    kt = kt.reshape(kt.shape[0])
    Ft = F.copy()
    atrow = np.zeros((t, p, 1))
    rtrow = np.zeros((t, p, p))
    gtrow = np.zeros((t, p, p))
    ctrow = np.zeros((t, p, p))
    wtrow = np.zeros((t, p, p))
    rtmat = np.zeros((t, p, p))
    theta = np.zeros((t, p))

    ## Forward Filtering

    for i in range(t):

        kt[i] = k_t
        if k_t == 1:
            Gt = G.copy()
            Wt = W.copy()
            rtrow[i] = np.eye(3)


        else:
            Gt = np.identity(3)
            #Gt[0, 0] = 0
            Wt = W.copy()
            #Gt = G.copy()
            Wt[1:,1:] = np.eye(2) * 0
        if i == 0:
            Gt = np.identity(3) #* 0
        ######################################################
        ### predict theta t|y-1
        at = Gt.dot(m_t)
        Rt = Gt.dot(Ct).dot(Gt.T) + Wt

        sft = Ft.dot(at)
        Qt = Ft.dot(Rt).dot(Ft.T) + V
        Qti = np.linalg.inv(Qt)
        KGt = Rt.dot(Ft.T).dot(Qti)
        m_t = at + KGt.dot(data[i].reshape((cmn, 1)) - sft)

        atrow[i] = at.copy()
        rtrow[i] = Rt.copy()
        gtrow[i] = Gt.copy()
        # Ct = Rt - KGt.dot(Ft).dot(Rt)
        Ct = np.round(Rt - KGt.dot(Ft).dot(Rt), 10)
        mt[i] = m_t.transpose()

        ctrow[i] = Ct.copy()
        wtrow[i] = Wt.copy()

        if m_t[0] > r:
            k_t = 1
        else:
            k_t = 0
        """
        try:
            if Ct[1, 1] == 0:
                theta[i] = m_t.reshape(-1)
                theta[i, 0] += mvn([[Ct[0, 0]]])  # cholesky([H_t[0,0]], lower=True).dot(np.random.rand(1))
            else:
                theta[i] = m_t.reshape(-1) + mvn(
                    Ct)  # + cholesky(H_t, lower=True).dot(np.random.rand(p))#rtrow[i].dot()
        except:
            print(H_t, kt[i], i)
            1 + "a"
        """

    ## Back_Sampling
    H_t = ctrow[-1]
    h_t = mt[-1]
    for i in range(t - 1, -1, -1):

        try:
            if H_t[1, 1] == 0:
                theta[i] = h_t.reshape(-1)
                theta[i, 0] += mvn([[H_t[0, 0]]])  # cholesky([H_t[0,0]], lower=True).dot(np.random.rand(1))
            else:
                theta[i] = h_t.reshape(-1) + mvn(
                    H_t)  # + cholesky(H_t, lower=True).dot(np.random.rand(p))#rtrow[i].dot()
        except:
            print(H_t, kt[i], i)
            1 + "a"
        if i != 0:
            Gt = gtrow[i]
            a_t = atrow[i]
            m_t = mt[i - 1]
            C_t = ctrow[i - 1]
            R_tp1 = rtrow[(i)]

            CGRi = C_t.dot(Gt.T).dot(np.linalg.inv(R_tp1))
            h_t = m_t.reshape(p, 1) + CGRi.dot(theta[i].reshape(p, 1) - a_t.reshape(p, 1))
            H_t = C_t - CGRi.dot(Gt).dot(C_t)
            H_t = np.round(H_t, 10)
    



    return theta, kt


# Form a shared array and a lock, to protect access to shared memory.

k = demand[1].shape[1]
kp = k - 1
# ck = C.shape[1]
h = len(demand)

if k == 35:
    try:
        C = np.array(pd.read_csv(PATH + "c.csv").iloc[:, 2:])
    except:
        C = np.array(pd.read_csv("sims6/c.csv").iloc[:, 2:])
else:
    C = np.eye(k)

ck = C.shape[1]
ckp = ck - 1

psia = np.zeros((h, ck))
psid = {}
glist = list()
start = 0
ind = np.zeros((h, 2))  # np.array([0] * h * 2).reshape((h, 2))
psih = []
gamh = []

for i in range(h):
    # psid[i] = np.zeros((demand[i].shape[0], ck))
    ind[i, 0] = start
    ind[i, 1] = start + demand[i].shape[0]
    glist.append(range(start, start + demand[i].shape[0]))
    start = start + demand[i].shape[0]
# print(glist)
# print(ind)
# psi = shared_array((t,k))
# psi = np.zeros((t,k))#np.concatenate([psid[x] for x in sorted(psid)], 0) #+ 0.01

dems = demand[0]
pris = prices[0]

cg = np.eye(ck)

for i in range(1, h):
    dems = np.vstack((dems, demand[i]))
    pris = np.vstack((pris, prices[i]))

dems = np.array(dems).reshape((-1, k))
pris = np.array(pris).reshape((-1, k))

t = dems.shape[0]
psi = np.zeros((t, ck))  # np.concatenate([psid[x] for x in sorted(psid)], 0) #+ 0.01

if len(sys.argv) > 2:
    c = int(sys.argv[2])
else:
    c = 4

thin = 1
nmcmc = burnin + thin * mcmc

### 3.  Prior Setting

m_psi = 0
tau_psi = 1

m_gamma = 0
tau_gamma = 1

### 4. Initial Value

zdata = np.array([1.] * h).reshape((h, 1))
rankz = zdata.shape[1]

gu0 = np.zeros((ck, 1)) + 0.01
for i0 in range(1):
    gu0[i0, i0] = 1
gv0 = np.diag([100.] * 1)
gv0i = np.linalg.inv(gv0)
ss0 = 2
sr0 = 2
srn = sr0 + t

bu0 = np.zeros((ck, 2)) + 0.01
for i in range(2):
    bu0[i, i] = 1
bv0 = np.diag([100.] * 2)
bv0i = np.linalg.inv(bv0)

ds0 = 2
dr0 = 2
drn = dr0 + t

pu0 = np.zeros((1, 1))
pv0 = np.diag([100.] * 1)

pv0i = np.linalg.inv(pv0)
pv0iu0 = pv0i.dot(pu0)

##############vb
f0 = 1 + 2
f0n = f0 + h
g0 = np.diag([f0]) * 100

tpa = np.zeros(t) + 0.05
tpp = np.zeros(t) + 0.05
# tpa = np.diag([0.1] * k)
# tpp = np.diag([0.1] * kp)

gama = np.zeros((t, ck))
gamm = np.zeros((t, ck))
gams = np.zeros((t, ck))

gh = dc(gu0)

fa = np.zeros((t, 1))
fm = np.zeros((t, 1))
fs = np.zeros((t, 1))

sigma = np.diag([1.] * ck)
sigmai = np.linalg.inv(sigma)

### baseline
psia = np.zeros((t, ck))
psim = np.zeros((t, ck))
psis = np.zeros((t, ck))

bh = dc(bu0)

ga = np.zeros((t, 2))
gm = np.zeros((t, 2))
gs = np.zeros((t, 2))

vg = np.identity(2)   #* 0.01

delta = np.diag([1.] * ck)
deltai = np.linalg.inv(delta)

betaa = np.zeros((h, 2)) + 0.01
betam = np.zeros((h, 2))
betas = np.zeros((h, 2))

phi = np.zeros((1, 2))

vb = np.diag([1.] * 2)  #* 0.01
vbi = np.linalg.inv(vb)

m = prices[0].shape[0]
mn = m - 1

if ck!= 12:
    cmn = ck * 2 - 1
else:
    cmn = ck * 2
kl = 1 + 2

m0 = np.zeros((kl, 1))
c0 = np.identity(kl)  # * 100

# print("c0",c0)
abh = np.zeros((cmn, kl))
sdm = np.diag([1.] * cmn)
kt = np.ones(t)
# kt = np.array([1.]*t).reshape((t,1))
ktm = np.zeros(t)
kts = dc(kt)

r = np.zeros((h, 1))

lla = np.zeros((t, 1))
llb = np.zeros((t, 1))

rpsi = psi.dot(C.T)
rgama = gama.dot(C.T)

for i in range(t):
    lla[i] = mdcev(rpsi[i], rgama[i], dems[i], pris[i], SIG)
art = np.zeros((t, 2))

itea = np.array([0] * h * 2).reshape((h, 2))

# v = np.diag([0.1] * ck)
# iv = np.linalg.inv(v)

## psi_mu, gam_mu
gammu = np.zeros((h, ck))
gammum = np.zeros((h, ck))
gammus = np.zeros((h, ck))

gamhmu = np.zeros(ck)
gamhmum = np.zeros(ck)
gamhmus = np.zeros(ck)

psimu = np.zeros((h, ck))
psimum = np.zeros((h, ck))
psimus = np.zeros((h, ck))

psihmu = np.zeros(ck)
psihmum = np.zeros(ck)
psihmus = np.zeros(ck)

vgam = np.identity(ck)
vpsi = np.identity(ck)

executor = ProcessPoolExecutor()
start = time.time()

psilogm = np.zeros((t, ck))
gamlogm = np.zeros((t, ck))
psilogs = np.zeros((t, ck))
gamlogs = np.zeros((t, ck))

import pandas as pd

if __name__ == '__main__':

    # betaa = np.array(pd.read_csv(PATH + "betas.csv"))[:, 1:]
    init = True
    #psi = np.array(pd.read_csv(PATH + "psis.csv"))[:, 1:]  # + np.random.normal(())   # Remove Index
    #gama = np.array(pd.read_csv(PATH + "gams.csv"))[:, 1:]

    # init = False
    if init:
        try:
            psi = np.array(pd.read_csv(PATH + "psis.csv"))[:, 1:]  # + np.random.normal(())   # Remove Index
            gama = np.array(pd.read_csv(PATH + "gams.csv"))[:, 1:]
            betaa = np.array(pd.read_csv(PATH + "betas.csv"))[:, 1:]
            fa = np.array(pd.read_csv(PATH + "f.csv"))
            ga = np.array(pd.read_csv(PATH + "g.csv"))
            gh = np.array(pd.read_csv(PATH + "a.csv"))[:, 1:]  # Remove Index
            bh = np.array(pd.read_csv(PATH + "b.csv"))[:, 1:]  # Remove Index
            phi = np.array([[-0.5, 0.5]])
        except:
            psi = np.array(pd.read_csv('sims6/' + "psis.csv"))[:, 1:]  # Remove Index
            gama = np.array(pd.read_csv('sims6/' + "gams.csv"))[:, 1:]
            betaa = np.array(pd.read_csv('sims6/' + "betas.csv"))[:, 1:]
            fa = np.array(pd.read_csv('sims6/' + "f.csv"))
            ga = np.array(pd.read_csv('sims6/' + "g.csv"))
            gh = np.array(pd.read_csv('sims6/' + "a.csv"))[:, 1:]  # Remove Index
            bh = np.array(pd.read_csv('sims6/' + "b.csv"))[:, 1:]  # Remove Index

    tgh = gh.copy()
    tbh = bh.copy()
    for imcmc in range(nmcmc):
        if imcmc % 10 == 0:  # print(imcmc); print(bh); print(phi); print(vb)
            print("Iter: %i,    LML: %.1f" % (imcmc, lla.sum()))
            print("spend time:", time.time() - start)
            print('abh', abh)
            print('fa', np.concatenate([fa, ga], 1)[:40])
            # print("psim",phi,"\n",betaa)
            # print("simga",np.diag(sigma), "delta", np.diag(delta))
        # print("Iter: %i,    POS: %.1f" % (imcmc, llb.sum()))
        if imcmc % 100 == 0:
            logging.info("Iter: %i" % imcmc)
            logging.info("spend time: %.3f" % (time.time() - start))
            logging.info("lml:%.3f" % lla.sum())
            logging.info(phi)
            logging.info(abh)
            logging.info(psihmu)
            logging.info(gamhmu)

        start = time.time()
        itea = np.zeros((t, 2))
        zphi = zdata.dot(phi)
        futures = []

        for i in range(h):
            futures.append(
                executor.submit(rwmh, demand[i], prices[i], 1, lla[glist[i]], gama[glist[i], :], fa[glist[i], :],
                                psi[glist[i], :], ga[glist[i], :], kt[glist[i]], r[i], betaa[i, :].transpose(),
                                zphi[i, :].transpose(), bh, gh, abh, sigma, delta, vbi, tpa[glist[i]], tpp[glist[i]],
                                llb[glist[i]], sdm, gammu[i, :], vgam, gamhmu, psimu[i, :], vpsi, psihmu, i))

        for i in range(h):
            # alpha,f,psi,g,ll,ite,kt,betai.T,p,a
            gama[glist[i], :], fa[glist[i], :], psi[glist[i], :], ga[glist[i], :], lla[glist[i]], itea[glist[i], :], kt[
                glist[i]], betaa[i, :], gammu[i, :], psimu[i, :], llb[glist[i]] = futures[i].result()

        art += itea

        gamab = gama.copy()
        psib = psi.copy()
        for i in range(h):
            gamab[glist[i]] -= gammu[i]
            psib[glist[i]] -= psimu[i]

        # gamab = gama - np.vstack( [np.repeat(gammu[i], len(gg), 0) for i,gg in enumerate(glist)])
        # psib = psi - np.vstack( [ [np.repeat(psimu[i], len(gg), 0) for i,gg in enumerate(glist)], np.zeros((t, 1))])

        for i in range(ck):
            if i == 0:
                res1 = psib[:, i:(i + 1)] - ga.dot(bh[i:(i + 1), :].T)
                delta[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res1.T.dot(res1)) / 2).rvs()
                res2 = gamab[:, i:(i + 1)] - fa.dot(gh[i:(i + 1), :].T)
                sigma[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res2.T.dot(res2)) / 2).rvs()
            elif i == 1:
                bh[i, 0], delta[i, i] = bmreg1(psib[:, i:(i + 1)] - ga[:, 1:2], ga[:, 0:1], None,
                                               delta[i, i])
                gh[i], sigma[i, i] = bmreg1(gamab[:, i:(i + 1)], fa, None, sigma[i, i])
            else:
                # if i != k - 1:
                # bh[i, 0], delta[i, i] = bmreg1(psib[:, i:(i + 1)] - ga[:, 1:2], ga[:, 0:1], None,delta[i, i])
                if i != k - 1:
                    bh[i], delta[i, i] = bmreg1(psib[:, i:(i + 1)], ga, None, delta[i, i],)
                gh[i], sigma[i, i] = bmreg1(gamab[:, i:(i + 1)], fa, None, sigma[i, i])
        #bh[:,0] = tbh[:,0].copy()

        """ Fix to 0.01
        for i in range(ck):
            if i == 0:
                res1 = psib[:, i:(i + 1)] - ga.dot(bh[i:(i + 1), :].T)
                delta[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res1.T.dot(res1)) / 2).rvs()
                res2 = gamab[:, i:(i + 1)] - fa.dot(gh[i:(i + 1), :].T)
                sigma[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res2.T.dot(res2)) / 2).rvs()
            elif i == 1:
                bh[i, 0], delta[i, i] = bmreg1(psib[:, i:(i + 1)] - ga[:, 1:2], ga[:, 0:1], None,
                                               0.01)
                gh[i], sigma[i, i] = bmreg1(gamab[:, i:(i + 1)], fa, None, 0.01)
            else:
                if i != k - 1:
                    bh[i], delta[i, i] = bmreg(psib[:, i:(i + 1)], ga, None, 0.01)
                gh[i], sigma[i, i] = bmreg(gamab[:, i:(i + 1)], fa, None, 0.01)

        """
        """ Fix prior to True Value
        for i in range(ck):
            if i == 0:
                res1 = psib[:, i:(i + 1)] - ga.dot(bh[i:(i + 1), :].T)
                delta[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res1.T.dot(res1)) / 2).rvs()
                res2 = gamab[:, i:(i + 1)] - fa.dot(gh[i:(i + 1), :].T)
                sigma[i, i] = ss.invgamma(a=(0.5 + t) / 2, scale=(0.5 + res2.T.dot(res2)) / 2).rvs()
            elif i == 1:
                bh[i, 0], delta[i, i] = bmreg1(psib[:, i:(i + 1)] - ga[:, 1:2], ga[:, 0:1], None,
                                               delta[i, i], tbh[i, 0].reshape(1,1), np.eye(1) *0.01)
                gh[i], sigma[i, i] = bmreg1(gamab[:, i:(i + 1)], fa, None, sigma[i, i], tgh[i].reshape(1,1), np.eye(1) *0.01 )
            else:
                # if i != k - 1:
                bh[i], delta[i, i] = bmreg(psib[:, i:(i + 1)], ga, None, delta[i, i], tbh[i].reshape(2,1), np.eye(2) * 0.01)
                gh[i], sigma[i, i] = bmreg(gamab[:, i:(i + 1)], fa, None, sigma[i, i],tgh[i].reshape(1,1), np.eye(1) *0.01)
        """



        if FIX:
            sigma = np.identity(ck) * VV  # * 0.1
            delta = np.identity(ck) * VV  # * 0.1

        for i in range(1, ck):
            gamhmu[i], vgam[i, i] = bmreg1(gammu[:, i:(i + 1)], np.ones((h, 1)), None, vgam[i, i])

        # ************
        #for i in range(ck ):
        for i in range(ck-1):
            psihmu[i], vpsi[i, i] = bmreg1(psimu[:, i:(i + 1)], np.ones((h, 1)), None, vpsi[i, i])

        #gamhmu[:] = 0
        #psihmu[:] = 0

        # print(np.diag(sigma))
        # print(np.diag(delta))
        if  ck == 12:
            sdm = np.zeros((ck * 2, ck * 2))
            sdm[:ck, :ck] = sigma.copy()
            sdm[ck:, ck:] = delta.copy()

            abh[0:(ck), 0] = gh.reshape((ck))
            abh[ck:(ck * 2), 1:(1 + 2)] = bh[0:ck, :]
        else:
            sdm = np.zeros((ck * 2 - 1, ck * 2 - 1))
            sdm[:(ck-1), :(ck-1)] = sigma[:-1,:-1].copy()
            sdm[(ck-1):, (ck-1):] = delta.copy()

            abh[0:(ck), 0] = gh.reshape((ck))
            abh[ck:, 1:] = bh[0:(ck-1), :]

        ## phi & vb
        
        for j in range(2):
            phi[0, j], vb[j, j] = bmreg1(betaa[:, j].reshape((h, 1)), zdata, phi, vb[j,j])#vb[j,j])
        #phi = np.array([[-0.5, 0.5]])
        #vb = np.identity(2)  * 0.1
        vbi = np.linalg.inv(vb)

        # Update Shock


        if imcmc < burnin and imcmc % 10 == 0:
            t_a = art[:, 0] / 10
            t_p = art[:, 1] / 10
            tpa = tpa * np.sqrt(1 + (t_a - target_mh))
            tpp = tpp * np.sqrt(1 + (t_p - target_mh))
            art = np.zeros((t, 2))
            # print(tpp,tpa)

        if imcmc >= burnin: #Save
            jmcmc = int((imcmc - burnin) / thin)
            lmcmc = jmcmc - int((mcmc / 2))

            if lmcmc >= 0:
                psilogm += psi.copy()
                psilogs += psi ** 2
                gamlogm += gama.copy()
                gamlogs += gama ** 2

            gamm += gama.copy()
            gams += gama ** 2

            fm += fa.copy()
            fs += fa ** 2

            ## baseline
            psim += psi.copy()
            psis += psi ** 2

            gm += ga
            gs += ga ** 2

            betam += betaa.copy()
            betas += betaa ** 2

            ktm += kt

            gammum += gammu.copy()
            gammus += gammu ** 2

            psimum += psimu
            psimus += psimu ** 2

            with open(PATH_PSIG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.mean(psi, axis=0))
            with open(PATH_GAMG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.mean(gama, axis=0))

            # with open(PATH_FG, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(fa.reshape(-1))

            # with open(PATH_GG, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(ga.reshape(-1))
            with open(PATH_BHG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(bh.reshape(-1))
            with open(PATH_GHG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(gh.reshape(-1))
            with open(PATH_SIGMAG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.diag(sigma))
            with open(PATH_DELTAG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.diag(delta))
            with open(PATH_VBG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(vb.reshape(-1))

            # with open(PATH_LLG, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(lla.reshape(-1))
            with open(PATH_PHIG, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(phi.reshape(-1))

            with open(PATH_VGAM, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.diag(vgam))

            with open(PATH_GAMMU, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(gamhmu)

            # with open(PATH_GAMMUH, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(gammu.reshape(-1))

            with open(PATH_VPSI, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(np.diag(vpsi))

            with open(PATH_PSIMU, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(psihmu)

            # with open(PATH_PSIMUH, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(psimu.reshape(-1))

            # with open(PATH_BETAG, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(betaa.reshape(-1))

            # with open(PATH_KG, 'a', newline='') as f:
            #    writer = csv.writer(f)
            #    writer.writerow(kt.reshape(-1))

            if jmcmc % 100 == 0:
                np.savez(FN, psim=psim, psis=psis, ktm=ktm, fm=fm, fs=fs, gm=gm, gs=gs,
                         gamm=gamm, gams=gams, mcmc=mcmc,
                         betam=betam, betas=betas, glist=glist, dems=dems, pris=pris, ind=ind, gammum=gammum,
                         gammus=gammus, psimum=psimum, psimus=psimus, fa=fa, ga=ga, kt=kt, psi=psi, gamma=gama,
                         betaa=betaa,
                         bh=bh, gh=gh)

        # print("spend time:", time.time() - start)

    np.savez(FN, psim=psim, psis=psis, ktm=ktm, fm=fm, fs=fs, gm=gm, gs=gs,
             gamm=gamm, gams=gams, mcmc=mcmc,
             betam=betam, betas=betas, glist=glist, dems=dems, pris=pris, ind=ind, gammum=gammum, gammus=gammus,
             psimum=psimum, psimus=psimus, psilogm=psilogm, psilogs=psilogs, gamlogm=gamlogm, gamlogs=gamlogs, fa=fa,
             ga=ga)
    '''

    res = rmultireg(y=psia[:,0:kp],x=zdata,bbar=tu0,a=tv0,nu=pf0,v=pg0,n=1)
    print("B:",res["B"])
    print("Sigma:", res["Sigma"][0])
    print( xpnd(np.array([1,2,3,4,5,6]) ) )
    print(rwishart(nu=pf0,v=pg0))
    '''



In [1]:
# Functions
import numpy as np
from math import gamma, sqrt
from scipy.stats import invgamma
## Common Functions

# Likelihood

def mdcev(psid, gamd, dem, pri, sigma=1):
    x = np.array(dem)
    p = np.array(pri)
    k = x.shape[0]
    n = 1
    expgam = np.array(np.exp(gamd))
    temp = psid
    temp2 = expgam

    v = temp - np.log(x * temp2 + 1) - np.log(p)
    xp = x > 0

    vi = np.copy(v)
    xi = np.copy(x)
    mv = xp.sum()  ## 0 = col  1 = row

    lli = (vi[xp]/sigma).sum() - mv * np.log(np.exp(vi/sigma).sum()) + np.log(gamma(mv)) - np.log(sigma ** (mv-1))

    gamj = expgam[xp]
    pij = p[xp]
    xij = xi[xp]
    jacv = 1 / (xij + 1/gamj)  ##jacobian
    ll = lli + np.log(jacv).sum() + np.log((pij / jacv).sum())
    # print("sumll",sumll)
    return ll



def h2t(x, l):
    res = None
    for h, i in zip(x, l):
        if res is None:
            res = np.tile(h, (i, 1))
        else:
            res = np.concatenate([res, np.tile(h, (i, 1))], axis=0)
    return res


# Fast sampling random value from multivariate-normal distribution
def mvn(cov):
    L = np.linalg.cholesky(cov)
    z = np.random.standard_normal(len(cov))
    return np.dot(L, z)

# Fast Logpdf from MVN
def logpdf(x, mean, cov):
    # `eigh` assumes the matrix is Hermitian.
    vals, vecs = np.linalg.eigh(cov)
    logdet     = np.sum(np.log(vals))
    valsinv    = np.array([1./v for v in vals])
    # `vecs` is R times D while `vals` is a R-vector where R is the matrix
    # rank. The asterisk performs element-wise multiplication.
    U          = vecs * np.sqrt(valsinv)
    rank       = len(vals)
    dev        = x - mean
    # "maha" for "Mahalanobis distance".
    maha       = np.square(np.dot(dev, U)).sum()
    log2pi     = np.log(2 * np.pi)
    return -0.5 * (rank * log2pi + maha + logdet)

#cholesky
def chol(x):
    if (x.shape[1] == 1 and x.shape[0] == 1):
        x = np.array(np.sqrt(x)).reshape((1, 1))
    else:
        x = np.linalg.cholesky(x)
    return x

# xpnd function in R
def xpnd(x):
    dim = int((-1 + sqrt(1 + 8 * len(x))) / 2)
    new = np.zeros((dim, dim))

    inds = np.tril_indices_from(new)
    new[inds] = x
    new[(inds[1], inds[0])] = x
    return new

# Bayes Reg

def bmreg(y, x, theta=None, Lambda=None, u0=None, v0=None, f0=None, g0=None):
    n = y.shape[0]
    m = y.shape[1]
    k = x.shape[1]
    l = m * k
    if u0 is None: u0 = np.zeros((k, m))
    if v0 is None: v0 = np.eye(k) * 100
    if f0 is None: f0 = k
    if g0 is None: g0 = np.eye(m)

    try:
        lambdai = np.linalg.inv(Lambda)
    except:
        lambdai = 1 / Lambda

    v0i = np.linalg.inv(v0)
    var = np.linalg.inv(x.transpose().dot(x) * lambdai + v0i)
    mean = var.dot((x.transpose() * lambdai).dot(y) + v0i.dot(u0)).reshape(k)
    s_theta = (mean + mvn(var)).reshape(k,m)

    ## generate lambda
    res = y - x.dot(s_theta)
    s_lambda = invgamma(a=1 + n/2, scale=1 + (res.T.dot(res)) / 2).rvs()

    return [s_theta.reshape(k), s_lambda]

#
def bmreg1(y, x, theta=None, Lambda=None, u0=None, v0=None, e=True):
    n = y.shape[0]
    m = y.shape[1]
    k = x.shape[1]
    l = m * k
    if u0 is None: u0 = np.zeros((k, m))
    if v0 is None: v0 = 100
    lambdai = 1 / Lambda

    v0i = 1/ v0
    var = np.linalg.inv(x.T.dot(x) * lambdai + v0i)
    mean = var.dot((x.T * lambdai).dot(y) + u0*v0i).reshape(k)

    ## generate lambda
    if e:
        s_theta = (mean + mvn(var)).reshape(k, m)
        res = y - x.dot(s_theta)
        s_lambda = invgamma(a=0.5 + n/2, scale=0.5 + (res.T.dot(res)) / 2).rvs()

        return [s_theta.reshape(k), s_lambda]
    else:
        return mean + mvn(var)

# For multiple core
def divide_work(h, coren):
    part = []
    rest = h % coren
    o = int((h - rest) / coren)
    temp = 0
    for i in range(coren):
        if i == (coren - 1):
            temp = rest
        part.append(range(i * o, i * o + o + temp))
    return part